In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Data import ExperimentData, AntData
from os import sep as sep
import AnalysisFunctions as afuns
from functools import reduce
import os

In [2]:
root_path = r'Y:\Lior&Einav\Experiments'
exp_details = pd.read_excel(root_path + sep + 'Experiments_details.xlsx', engine='openpyxl')

In [3]:
def get_forager_filename(exp_path):
    forager_filename = [filename for filename in os.listdir(exp_path) if filename.startswith(r'forager_types')][0]
    return forager_filename

In [4]:
def get_forager_samples(forager,tdata,fdata):
    # forager's interactions
    forager_giving_interactions = tdata[tdata.giver==forager]
    forager_receiving_interactions = tdata[tdata.receiver==forager]
    renamed_g=forager_giving_interactions.rename(columns={"receiver_crop_before_red": "partner_crop_red", "receiver_crop_before_yellow": "partner_crop_yellow"})
    renamed_r=forager_receiving_interactions.rename(columns={"giver_crop_before_red": "partner_crop_red", "giver_crop_before_yellow": "partner_crop_yellow"})
    forager_interactions = pd.concat([renamed_g,renamed_r]).sort_values(by="start_frame")

    # forager's feedings
    forager_feedings = fdata[fdata.ant_id==forager].sort_values(by="feeding_start")

    # forager's samples between feedings
    forager_interactions['trip'] = pd.cut(forager_interactions.start_frame, forager_feedings.feeding_start,labels=False)
    mean_sample_red = forager_interactions.groupby('trip')['partner_crop_red'].mean()
    mean_sample_yellow = forager_interactions.groupby('trip')['partner_crop_yellow'].mean()
    
    return mean_sample_red, mean_sample_yellow, forager_feedings

In [5]:
def get_colony_state_per_trip(crops,conversion_factors,forager_feedings):
    
    colony_red = crops['red'].loc[forager_feedings.feeding_start].mean(axis=1)
    colony_yellow = crops['yellow'].loc[forager_feedings.feeding_start].mean(axis=1)
    colony_red = colony_red/conversion_factors['red'][0]
    colony_red = colony_red.reset_index(name="colony_red")
    colony_yellow = colony_yellow/conversion_factors['yellow'][0]
    colony_yellow = colony_yellow.reset_index(name="colony_yellow")
    
    return colony_red, colony_yellow

In [6]:
def get_forager_sample_vs_colony(forager, tdata, fdata, crops, conversion_factors, return_forager_feedings=False):
    
    mean_sample_red, mean_sample_yellow, forager_feedings = get_forager_samples(forager,tdata,fdata)
    colony_red, colony_yellow = get_colony_state_per_trip(crops,conversion_factors,forager_feedings)
    
    df = reduce(lambda left,right: pd.merge(left, right, left_index=True, right_index=True, how='outer'), 
                [colony_red,colony_yellow,mean_sample_red,mean_sample_yellow])
    
    if return_forager_feedings:
        return df, forager_feedings
    
    return df

In [7]:
def add_feeding_data(df,forager_feedings):
    df['previous_food_source']=forager_feedings.reset_index()['food_source']
    df['next_food_source']=forager_feedings.reset_index()['food_source'].shift(-1)
    df['start_frame']=forager_feedings.reset_index()['feeding_start']
    return df

# Forager's sample vs. Colony

## One colony

### Get experiment data

In [8]:
exp_num = 20
exp_idx, PC_ratios_dict, exp_path = afuns.get_exp_data(exp_num, exp_details, root_path)

fdata = pd.read_csv(exp_path + sep + r'forager_table_with_feeding_sizes_ul_transparency_corrected.csv', index_col=[0])
tdata = pd.read_csv(exp_path + sep + r'clean_trophallaxis_table_transparency_corrected.csv')
crops = pd.read_csv(exp_path + sep + r'clean_crops_transparency_corrected.csv', header=[0,1], index_col=[0]).swaplevel(axis=1)
ants = pd.read_csv(exp_path + sep + r'ants_list.csv')
conversion_factors = pd.read_csv(exp_path + sep + r'conversion_factors_by_weight_and_feeding_sum.csv')
foragers = pd.read_csv(exp_path + sep + get_forager_filename(exp_path))
conversion_factors

,red,yellow
0,55073.158583,42053.272903


In [9]:
tdata.head()

,giver,receiver,start_frame,end_frame,transferred_red,transferred_yellow,x,y,giver_crop_before_red,giver_crop_before_yellow,receiver_crop_before_red,receiver_crop_before_yellow,estimation_confidence,group_id
0,56,424,67.0,68.0,0.000000,0.0,1157.093768,2202.898169,0.000000,0.0,0.000000,0.000000,3,NaN
1,1124,236,162.0,177.0,4.545942,0.0,1157.271529,1865.089656,16.823011,0.0,0.000000,0.000000,3,1.0
2,1292,236,175.0,177.0,0.969859,0.0,NaN,NaN,0.000000,0.0,4.202721,0.300993,1,1.0
3,1124,1292,175.0,206.0,4.066397,0.0,NaN,NaN,15.263475,0.0,0.000000,0.000000,3,1.0
4,1235,1230,179.0,287.0,9.204103,0.0,1790.034667,1040.514318,21.739250,0.0,0.000000,0.000000,3,2.0


In [10]:
fdata.head()

,ant_id,crop_after,crop_before,feeding_end,feeding_size_intensity,feeding_start,first_interaction_after_partner,first_interaction_after_start,food_source,last_interaction_before_end,last_interaction_before_partner,feeding_size_ul
0,386,425459.750000,0.000000,496,425459.750000,399,NaN,NaN,yellow,NaN,NaN,11.956357
1,1299,291001.127769,45089.202515,837,245911.925255,474,NaN,NaN,yellow,NaN,NaN,5.803276
2,673,15474.659424,11099.045541,835,4375.613882,833,NaN,NaN,yellow,NaN,NaN,0.196291
3,386,344788.868750,2241.443140,1189,342547.425610,920,NaN,NaN,yellow,NaN,NaN,9.626338
4,1299,0.000000,7084.583008,1355,0.000000,1336,NaN,NaN,yellow,NaN,NaN,0.000000


In [11]:
ants

,ant_id,is_forager
0,1004,False
1,1063,False
2,1068,False
3,1079,False
4,1117,False
5,1230,False
6,1299,True
7,1329,False
8,1354,False
9,1355,False


In [12]:
foragers

,forager_id,num_feedings,num_red_feedings,num_yellow_feedings,forager_type
0,1299,7,4,3,mix_forager
1,236,4,1,3,taster
2,386,9,5,4,mix_forager
3,424,2,2,0,taster
4,61,8,2,6,protein_forager
5,1124,13,9,4,sugar_forager
6,1235,18,13,5,sugar_forager
7,150,3,2,1,taster
8,673,0,0,0,taster


## Per forager (by forager type)

In [13]:
def plot_foragers_sample_vs_colony(ax,forager, tdata, fdata, crops, conversion_factors, foragers, bin_data=True):


    df =  get_forager_sample_vs_colony(forager, tdata, fdata, crops, conversion_factors)
    
    # plot
    if bin_data:
        # bin data
        df['yellow_col_state_bin'] = pd.cut(df['colony_yellow'],5)
        grouped_yellow_mean = df[{'partner_crop_yellow','colony_yellow'}].groupby(df['yellow_col_state_bin']).mean()
        grouped_yellow_std = df[{'partner_crop_yellow','colony_yellow'}].groupby(df['yellow_col_state_bin']).std()
        df['red_col_state_bin'] = pd.cut(df['colony_red'],5)
        grouped_red_mean = df[{'partner_crop_red','colony_red'}].groupby(df['red_col_state_bin']).mean()
        grouped_red_std = df[{'partner_crop_red','colony_red'}].groupby(df['red_col_state_bin']).std()

        # binned plot
        ax.axline((0,0),slope=1,color='k')
        j1=ax.errorbar(grouped_red_mean['colony_red'],grouped_red_mean['partner_crop_red'],
                     xerr=grouped_red_std['colony_red'],yerr=grouped_red_std['partner_crop_red'],fmt='r')
        j2=ax.errorbar(grouped_yellow_mean['colony_yellow'],grouped_yellow_mean['partner_crop_yellow'],
                     xerr=grouped_yellow_std['colony_yellow'],yerr=grouped_yellow_std['partner_crop_yellow'],fmt='g')
        ax.set_xlabel('Colony State [ul]')
        ax.set_ylabel('Mean partner''s crop [ul]')
        ax.legend([j1,j2], ['Sugar-rich','Protein-rich'])
        ax.set_title(f'Forager {forager} {foragers.forager_type[foragers.forager_id==forager].iloc[0]}')
    else:
        # scatter plot
        ax.axline((0,0),slope=1,color='k')
        j1,  = ax.plot(df.colony_red, df.partner_crop_red,'.r')
        j2,  = ax.plot(df.colony_yellow, df.partner_crop_yellow,'.g')
        ax.set_xlabel('Colony State [ul]')
        ax.set_ylabel('Mean partner''s crop [ul]')
        ax.legend([j1,j2], ['Sugar-rich','Protein-rich'])
        ax.set_title(f'Forager {forager} {foragers.forager_type[foragers.forager_id==forager].iloc[0]}')

In [14]:
sugar_foragers = foragers['forager_id'][foragers.forager_type=='sugar_forager'].values
protein_foragers = foragers['forager_id'][foragers.forager_type=='protein_forager'].values
mix_foragers = foragers['forager_id'][foragers.forager_type=='mix_forager'].values
mix_foragers

array([1299,  386], dtype=int64)

In [15]:
foragers.sort_values(by='forager_type')

,forager_id,num_feedings,num_red_feedings,num_yellow_feedings,forager_type
0,1299,7,4,3,mix_forager
2,386,9,5,4,mix_forager
4,61,8,2,6,protein_forager
5,1124,13,9,4,sugar_forager
6,1235,18,13,5,sugar_forager
1,236,4,1,3,taster
3,424,2,2,0,taster
7,150,3,2,1,taster
8,673,0,0,0,taster


In [16]:
%matplotlib qt
fig, axes = plt.subplots(3, max(len(protein_foragers),len(sugar_foragers),len(mix_foragers)))
counter = 0
for forager in protein_foragers:
    ax = axes[0,counter]
    plot_foragers_sample_vs_colony(ax,forager, tdata, fdata, crops, conversion_factors, foragers)
    counter+=1
counter = 0
for forager in sugar_foragers:
    ax = axes[1,counter]
    plot_foragers_sample_vs_colony(ax,forager, tdata, fdata, crops, conversion_factors, foragers)
    counter+=1
counter = 0
for forager in mix_foragers:
    ax = axes[2,counter]
    plot_foragers_sample_vs_colony(ax,forager, tdata, fdata, crops, conversion_factors, foragers)
    counter+=1

## Pooled data

In [21]:
forager_dfs=[]
consistent_foragers = foragers[foragers['forager_type']!='taster']
for forager in consistent_foragers['forager_id']:
    df, forager_feedings = get_forager_sample_vs_colony(forager, tdata, fdata, crops, 
                                                        conversion_factors, return_forager_feedings=True)
    df = add_feeding_data(df,forager_feedings)
    forager_dfs.append(df)
pooled_df = pd.concat(forager_dfs)
pooled_df

,index_x,colony_red,index_y,colony_yellow,partner_crop_red,partner_crop_yellow,previous_food_source,next_food_source,start_frame
0,55,0.083088,55,0.000000,6.846918,0.155427,red,yellow,55
1,474,1.246510,474,0.265739,NaN,NaN,yellow,red,474
2,905,3.067146,905,0.519520,8.242960,0.025669,red,yellow,905
3,1336,3.727560,1336,0.710244,12.056925,11.641606,yellow,yellow,1336
4,1534,4.216965,1534,0.829791,3.598048,3.893593,yellow,red,1534
...,...,...,...,...,...,...,...,...,...
15,6739,9.522767,6739,2.247584,NaN,NaN,yellow,yellow,6739
16,6829,9.753025,6829,2.232707,10.348442,4.206032,yellow,yellow,6829
17,7719,10.324637,7719,2.888934,14.797672,7.847594,yellow,yellow,7719
18,9883,10.749046,9883,3.088277,NaN,NaN,yellow,red,9883


In [46]:
def bin_data(df, data_column, by, num_bins):
    bin_column = pd.cut(df[by],num_bins)
    data_mean = df[{data_column,by}].groupby(bin_column).mean()
    data_std = df[{data_column,by}].groupby(bin_column).std(ddof=0)
    sample_size = df[data_column].groupby(bin_column).size()
    return data_mean, data_std, sample_size


In [22]:
pooled_df['yellow_col_state_bin'] = pd.cut(pooled_df['colony_yellow'],5)
grouped_yellow_mean = pooled_df[{'partner_crop_yellow','colony_yellow'}].groupby(pooled_df['yellow_col_state_bin']).mean()
grouped_yellow_std = pooled_df[{'partner_crop_yellow','colony_yellow'}].groupby(pooled_df['yellow_col_state_bin']).std()
pooled_df['red_col_state_bin'] = pd.cut(pooled_df['colony_red'],5)
grouped_red_mean = pooled_df[{'partner_crop_red','colony_red'}].groupby(pooled_df['red_col_state_bin']).mean()
grouped_red_std = pooled_df[{'partner_crop_red','colony_red'}].groupby(pooled_df['red_col_state_bin']).std()

# binned plot
plt.figure()
plt.axline((0,0),slope=1,color='k')
j1=plt.errorbar(grouped_red_mean['colony_red'],grouped_red_mean['partner_crop_red'],
             xerr=grouped_red_std['colony_red'],yerr=grouped_red_std['partner_crop_red'],fmt='r')
j2=plt.errorbar(grouped_yellow_mean['colony_yellow'],grouped_yellow_mean['partner_crop_yellow'],
             xerr=grouped_yellow_std['colony_yellow'],yerr=grouped_yellow_std['partner_crop_yellow'],fmt='g')
plt.xlabel('Colony State [ul]')
plt.ylabel('Mean partner''s crop [ul]')
plt.legend([j1,j2], ['Sugar-rich','Protein-rich'])

In [23]:
next_yellow = pooled_df[pooled_df['next_food_source']=='yellow']
next_red =  pooled_df[pooled_df['next_food_source']=='red']

In [24]:
next_yellow['yellow_col_state_bin'] = pd.cut(next_yellow['colony_yellow'],5)
next_yellow_yellow_mean = next_yellow[{'partner_crop_yellow','colony_yellow'}].groupby(next_yellow['yellow_col_state_bin']).mean()
next_yellow_yellow_std = next_yellow[{'partner_crop_yellow','colony_yellow'}].groupby(next_yellow['yellow_col_state_bin']).std()
next_yellow['red_col_state_bin'] = pd.cut(next_yellow['colony_red'],5)
next_yellow_red_mean = next_yellow[{'partner_crop_red','colony_red'}].groupby(next_yellow['red_col_state_bin']).mean()
next_yellow_red_std = next_yellow[{'partner_crop_red','colony_red'}].groupby(next_yellow['red_col_state_bin']).std()

# binned plot
plt.figure()
plt.axline((0,0),slope=1,color='k')
j1=plt.errorbar(next_yellow_red_mean['colony_red'],next_yellow_red_mean['partner_crop_red'],
             xerr=next_yellow_red_std['colony_red'],yerr=next_yellow_red_std['partner_crop_red'],fmt='r')
j2=plt.errorbar(next_yellow_yellow_mean['colony_yellow'],next_yellow_yellow_mean['partner_crop_yellow'],
             xerr=next_yellow_yellow_std['colony_yellow'],yerr=next_yellow_yellow_std['partner_crop_yellow'],fmt='g')
plt.xlabel('Colony State [ul]')
plt.ylabel('Mean partner''s crop [ul]')
plt.legend([j1,j2], ['Sugar-rich','Protein-rich'])
plt.title('next feeding protein')

C:\Users\liorba\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\liorba\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Text(0.5, 1.0, 'next feeding protein')

In [25]:
next_red['yellow_col_state_bin'] = pd.cut(next_red['colony_yellow'],5)
next_red_yellow_mean = next_red[{'partner_crop_yellow','colony_yellow'}].groupby(next_red['yellow_col_state_bin']).mean()
next_red_yellow_std = next_red[{'partner_crop_yellow','colony_yellow'}].groupby(next_red['yellow_col_state_bin']).std()
next_red['red_col_state_bin'] = pd.cut(next_red['colony_red'],5)
next_red_red_mean = next_red[{'partner_crop_red','colony_red'}].groupby(next_red['red_col_state_bin']).mean()
next_red_red_std = next_red[{'partner_crop_red','colony_red'}].groupby(next_red['red_col_state_bin']).std()

# binned plot
plt.figure()
plt.axline((0,0),slope=1,color='k')
j1=plt.errorbar(next_red_red_mean['colony_red'],next_red_red_mean['partner_crop_red'],
             xerr=next_red_red_std['colony_red'],yerr=next_red_red_std['partner_crop_red'],fmt='r')
j2=plt.errorbar(next_red_yellow_mean['colony_yellow'],next_red_yellow_mean['partner_crop_yellow'],
             xerr=next_red_yellow_std['colony_yellow'],yerr=next_red_yellow_std['partner_crop_yellow'],fmt='g')
plt.xlabel('Colony State [ul]')
plt.ylabel('Mean partner''s crop [ul]')
plt.legend([j1,j2], ['Sugar-rich','Protein-rich'])
plt.title('next feeding sugar')

C:\Users\liorba\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\liorba\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


Text(0.5, 1.0, 'next feeding sugar')